import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate0=80
rateNet=480
cls_num=2
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=DistalPhalanxOutlineCorrect0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

first_train_shape= (276, 81)
first_test_shape= (600, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [161]
max(train_feature_Altitude)= 2.4602
min(train_feature_Altitude)= -2.1799
first_train_sample=
 [ 1.       -0.88425  -0.76511  -0.6264   -0.30459  -0.014302  0.29922
  0.51262   0.62572   0.89455   1.053     1.1322    1.2672    1.4321
  1.4845    1.2757    1.1589    1.0053    0.81736   0.50439   0.37885
  0.14581  -0.20947  -0.57872  -0.67481  -0.79999  -0.93504  -0.78816
 -0.57744  -0.27191   0.061242  0.41037   0.76072   1.1348    1.5046
  1.6696    1.6999    1.5531    1.3937    1.4704    1.5877    1.6902
  1.7647    1.5246    1.1461    0.76139   0.42848   0.076242 -0.2335
 -0.51671  -0.81834  -0.080823 -0.27898  -0.46208  -0.69433  -0.92273
 -1.1133   -1.3737   -1.5067   -1.5621   -1.3708   -1.102    -0.86298
 -0.53403  -0.3252   -0.1899   -0.3366   -0.45422  -0.60653  -0.78383
 -1.0497   -1.3447   -1.4977   -1.5734   -1.4749   -1.2418   -1.0149
 -0.78176  -0.61354  -0.306

In [3]:
print(np.shape(ecg))

(276, 81)


<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=rate0                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.empty((0,smpl_rte+1))

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    windws=np.append(windws,vars()['wndws'+str(cls)],axis=0)
    
print("each class and its windows =")

furir_windws0=np.fft.fft(np.array(windws[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([windws[:,-1]])),axis=1)
print("shape furir_windws=", np.shape(furir_windws))

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
shape furir_windws0= (276, 80)
shape furir_windws= (276, 81)
0 115	1 161	
 max = 161


In [5]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(1.5*mx_wndws)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

241


<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Random CNN2</div></center></h1>

In [6]:
rpt_cnn_clsfctn=1
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 


for i in range (0,cls_num):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (0,cls_num):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],scl_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrp_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDO_rndm_beat(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqnc_rndm_beat(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (0,cls_num):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (0,cls_num):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_train=np.array(ytrain)
            X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
            y_test=np.array(ytest)
            X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_valid=np.array(ytrain)

            if min(y_train)==1:
                y_train=y_train-1
                y_test=y_test-1
                y_valid =y_valid-1            

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            #VGG16_1D
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data

            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))

            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
            
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\Users\user\Thesis\Classifying\CNNsplit\CNN_DistalPhalanxOutlineCorrect78_77_strt0\augment.py:658: ComplexWarning: Casting complex values to real discards the imaginary part
  p_wn = np.float16(np.concatenate((P,pl), axis=1))


19/19 [==============================] - 0s 4ms/step


In [7]:
rslts

array(['mthd 1  Mgntud 0  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 1  Mgntud 1  average= [77 77 78]  max= [77 77 78]  var= 0',
       'mthd 1  Mgntud 2  average= [76 75 76]  max= [76 75 76]  var= 0',
       'mthd 2  Mgntud 0  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 2  Mgntud 1  average= [76 74 76]  max= [76 74 76]  var= 0',
       'mthd 2  Mgntud 2  average= [76 74 76]  max= [76 74 76]  var= 0',
       'mthd 3  Mgntud 0  average= [78 77 78]  max= [78 77 78]  var= 0',
       'mthd 3  Mgntud 1  average= [75 74 76]  max= [75 74 76]  var= 0',
       'mthd 3  Mgntud 2  average= [76 76 77]  max= [76 76 77]  var= 0',
       'mthd 4  Mgntud 0  average= [78 77 78]  max= [78 77 78]  var= 0',
       'mthd 4  Mgntud 1  average= [76 75 77]  max= [76 75 77]  var= 0',
       'mthd 4  Mgntud 2  average= [76 75 77]  max= [76 75 77]  var= 0',
       'mthd 5  Mgntud 0  average= [76 75 76]  max= [76 75 76]  var= 0',
       'mthd 5  Mgntud 1  average= [77 75 77]  max=

<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Marginal CNN2</div></center></h1>

In [8]:
rpt_cnn_clsfctn=1
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (0,cls_num):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (0,cls_num):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],sclRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],sclRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],sclRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDORaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDORaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDORaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (0,cls_num):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (0,cls_num):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_train=np.array(ytrain)
            X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
            y_test=np.array(ytest)
            X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
            y_valid=np.array(ytrain)

            if min(y_train)==1:
                y_train=y_train-1
                y_test=y_test-1
                y_valid =y_valid-1            

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

            predict_x=model_crs.predict(X_test)                # Function 1
            y_pred_crs=np.argmax(predict_x,axis=1)             # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\Thesis\Classifying\CNNsplit\CNN_DistalPhalanxOutlineCorrect78_77_strt0\augment.py:1504: ComplexWarning: Casting complex values to real discards the imaginary part
  return(np.float16(p_aug_raw))


19/19 [==============================] - 0s 4ms/step


In [9]:
rslts

array(['mthd 1  Mgntud 0  average= [76 74 76]  max= [76 74 76]  var= 0',
       'mthd 1  Mgntud 1  average= [74 73 74]  max= [74 73 74]  var= 0',
       'mthd 1  Mgntud 2  average= [76 74 76]  max= [76 74 76]  var= 0',
       'mthd 2  Mgntud 0  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 2  Mgntud 1  average= [79 77 79]  max= [79 77 79]  var= 0',
       'mthd 2  Mgntud 2  average= [74 73 74]  max= [74 73 74]  var= 0',
       'mthd 3  Mgntud 0  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 3  Mgntud 1  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 3  Mgntud 2  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 4  Mgntud 0  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 4  Mgntud 1  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 4  Mgntud 2  average= [77 76 77]  max= [77 76 77]  var= 0',
       'mthd 5  Mgntud 0  average= [78 77 78]  max= [78 77 78]  var= 0',
       'mthd 5  Mgntud 1  average= [75 74 75]  max=